# Wrangling - Looking Into How Parks in Paradise Compare to Cities of Similar Size

**I explore how the number of mobile home lots in Paradise compare to other cities with a similar population in 2018 (before the Camp Fire).** 

To answer the above question, I will use two notebooks. This one is focused on **wrangling and merging**. The notebook called 'MH_paradise_analysis' is focused on **data analysis**. 

There are two relevant datasets:
* List of mobile home parks permitted by the California Housing and Community Development Department. [Source.]('https://casas.hcd.ca.gov/casas/cmirMp/onlineQuery')
* Population by city and town in California. [Source.]('https://www.census.gov/programs-surveys/popest/technical-documentation/research/evaluation-estimates/2020-evaluation-estimates/2010s-cities-and-towns-total.html')

I will start by importing necessary libraries and creating an folder paths for the datasets I've cleaned and would like to import/export.

In [1]:
import pandas as pd 
import numpy as np
import os
#import pandas and numpy libraries

In [2]:
raw_dir = '/Users/kavishharjai/code/mh_paradise/raw/'
processed_outdir = '/Users/kavishharjai/code/mh_paradise/processed'

#### Cleaning MH dataset

In [3]:
list_of_parks = pd.read_csv(raw_dir + 'MH_parks.csv')
list_of_parks.head()

,Park Name,County,Park Identifier,Park Address,Fire Authority,Jurisdiction,MH Spaces,RV Lots W/Drains,RV Lots W/O Drains,Operated by,ADDRESS
0,SPANISH RANCH II,ALAMEDA,01-0001-MP,"121 RANCHERO WAY, HAYWARD, CA 94544, (510) 886...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",187.0,0.0,0.0,"HOMETOWN SPANISH RANCH, LLC, A DELAWARE LIMITE...","121 RANCHERO WAY, HAYWARD, CA 94544, (510) 886..."
1,GEORGIAN MANOR MHP,ALAMEDA,01-0003-MP,"1419 BUCKINGHAM WAY, HAYWARD, CA 94544, (510) ...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",265.0,0.0,0.0,"GEORGIAN MANOR MOBILEHOME PARK L P, 321 HARTZ ...","1419 BUCKINGHAM WAY, HAYWARD, CA 94544, (510) ..."
2,BAL TRAILER CT,ALAMEDA,01-0008-MP,"14831 BANCROFT AVE, SAN LEANDRO, CA 94578, (51...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",31.0,0.0,0.0,"PUEBLO SPRINGS MHP LTD, 10351 SANTA MONICA BLV...","14831 BANCROFT AVE, SAN LEANDRO, CA 94578, (51..."
3,VASCO MHP,ALAMEDA,01-0010-MP,"6539 S FRONT RD, LIVERMORE, CA 94550, (925) 44...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",50.0,21.0,0.0,"SOLOMON/MANDEL, EDWARD/GEORGE, PO BOX 406, ALA...","6539 S FRONT RD, LIVERMORE, CA 94550, (925) 44..."
4,AVALON MOBILE HOME PARK,ALAMEDA,01-0013-MP,"3970 CASTRO VALLEY BLVD, CASTRO VALLEY, CA 945...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",49.0,0.0,0.0,"AVALON MHP INC, 4061 EAST CASTRO VALLEY BLVD S...","3970 CASTRO VALLEY BLVD, CASTRO VALLEY, CA 945..."


**I will start by standardizing column names so there aren't spaces between words.**

In [4]:
list_of_parks = list_of_parks.rename(columns = {'Park Name': 'park_name',
                                                'County':'county',
                                              'Park Identifier': 'park_identifier', 
                                              'Park Address': 'park_address', 
                                              'MH Spaces': 'mh_spaces', 
                                              'Fire Authority': 'fire_authority', 
                                              'Jurisdiction': 'jurisdiction',
                                             'RV Lots W/Drains': 'rv_lots_drains',
                                             'RV Lots W/O Drains': 'rv_lots_no_drains',
                                             'Operated by': 'operated_by',
                                             'ADDRESS': 'operater_address'})

**In addition to mobile homes, this dataset contains information on RV Parks.**

**Since my analysis is focused on mobile homes, I will remove all observations where the value for MH_Spaces is 0.**

In [5]:
list_of_parks = list_of_parks[list_of_parks.mh_spaces != 0]

In [6]:
len(list_of_parks)

4668

**After filtering my dataframe, I wanted to check out what kind of data types there are and whether there are columns that are missing a bunch of values.** 

In [7]:
list_of_parks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4668 entries, 0 to 5247
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   park_name          4668 non-null   object 
 1   county             4668 non-null   object 
 2   park_identifier    4668 non-null   object 
 3   park_address       4668 non-null   object 
 4   fire_authority     4668 non-null   object 
 5   jurisdiction       4668 non-null   object 
 6   mh_spaces          4667 non-null   float64
 7   rv_lots_drains     4667 non-null   float64
 8   rv_lots_no_drains  4667 non-null   float64
 9   operated_by        4664 non-null   object 
 10  operater_address   4668 non-null   object 
dtypes: float64(3), object(8)
memory usage: 437.6+ KB


**It looks pretty good, but since I'm invested in checking out the amount of spaces in each park, I want to probably get rid of the observation where the value of MH_Spaces is null.**

In [8]:
list_of_parks[list_of_parks['mh_spaces'].isnull()]

,park_name,county,park_identifier,park_address,fire_authority,jurisdiction,mh_spaces,rv_lots_drains,rv_lots_no_drains,operated_by,operater_address
2982,8456 BRADSHAW ROAD (S.O.P. W/OUT P.T.O.),SACRAMENTO,34-13586-MP,"8456 BRADSHAW ROAD, ELK GROVE, CA 95624","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",NaN,NaN,NaN,NaN,"8456 BRADSHAW ROAD, ELK GROVE, CA 95624"


In [9]:
list_of_parks = list_of_parks.drop(labels=2982, axis=0)

**The 'park_address' column annoyingly contains the park phone number. Let me split that column into separate entities, and then rejoin on all the address components. I ultimately will only be interested in the city column.** 

In [10]:
list_of_parks['park_address'] = list_of_parks.park_address.str.title()

In [11]:
list_of_parks.head(10)

,park_name,county,park_identifier,park_address,fire_authority,jurisdiction,mh_spaces,rv_lots_drains,rv_lots_no_drains,operated_by,operater_address
0,SPANISH RANCH II,ALAMEDA,01-0001-MP,"121 Ranchero Way, Hayward, Ca 94544, (510) 886...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",187.0,0.0,0.0,"HOMETOWN SPANISH RANCH, LLC, A DELAWARE LIMITE...","121 RANCHERO WAY, HAYWARD, CA 94544, (510) 886..."
1,GEORGIAN MANOR MHP,ALAMEDA,01-0003-MP,"1419 Buckingham Way, Hayward, Ca 94544, (510) ...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",265.0,0.0,0.0,"GEORGIAN MANOR MOBILEHOME PARK L P, 321 HARTZ ...","1419 BUCKINGHAM WAY, HAYWARD, CA 94544, (510) ..."
2,BAL TRAILER CT,ALAMEDA,01-0008-MP,"14831 Bancroft Ave, San Leandro, Ca 94578, (51...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",31.0,0.0,0.0,"PUEBLO SPRINGS MHP LTD, 10351 SANTA MONICA BLV...","14831 BANCROFT AVE, SAN LEANDRO, CA 94578, (51..."
3,VASCO MHP,ALAMEDA,01-0010-MP,"6539 S Front Rd, Livermore, Ca 94550, (925) 44...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",50.0,21.0,0.0,"SOLOMON/MANDEL, EDWARD/GEORGE, PO BOX 406, ALA...","6539 S FRONT RD, LIVERMORE, CA 94550, (925) 44..."
4,AVALON MOBILE HOME PARK,ALAMEDA,01-0013-MP,"3970 Castro Valley Blvd, Castro Valley, Ca 945...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",49.0,0.0,0.0,"AVALON MHP INC, 4061 EAST CASTRO VALLEY BLVD S...","3970 CASTRO VALLEY BLVD, CASTRO VALLEY, CA 945..."
5,NEW ENGLAND VILLAGE,ALAMEDA,01-0015-MP,"940 New England Village Dr, Hayward, Ca 94544,...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",415.0,0.0,0.0,"BRANDENBURG STAEDLER & MOORE, 1122 WILLOW ST S...","940 NEW ENGLAND VILLAGE DR, HAYWARD, CA 94544,..."
6,DEL RIO MOBILE HOME,ALAMEDA,01-0016-MP,"1550 162Nd Ave, San Leandro, Ca 94578, (510) 2...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",52.0,0.0,0.0,"CALIFANO FAMILY TRUST, 1550 162ND AVENUE SPACE...","1550 162ND AVE, SAN LEANDRO, CA 94578, (510) 2..."
7,DEL VALLE MHP,ALAMEDA,01-0017-MP,"1148 Arroyo Rd, Livermore, Ca 94550, (510) 331...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",29.0,0.0,0.0,"ABBOUD FAMILY TRUST, PO BOX 3975, HAYWARD, CA ...","1148 ARROYO RD, LIVERMORE, CA 94550, (510) 331..."
8,BAYSHORE COMMONS,ALAMEDA,01-0023-MP,"1468 Grand Ave, San Leandro, Ca 94577, (510) 3...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",40.0,0.0,0.0,"SAN LEANDRO TRAILER PARK LLC, 6653 EMBARCADERO...","1468 GRAND AVE, SAN LEANDRO, CA 94577, (510) 3..."
9,FAIRVIEW TP,ALAMEDA,01-0024-MP,"785 Rose Ave, Pleasanton, Ca 94566, (925) 846-...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",14.0,8.0,0.0,"GIL, EDMUND, 2606 DERBY DR, SAN RAMON, CA 94583","785 ROSE AVE, PLEASANTON, CA 94566, (925) 846-..."


In [12]:
list_of_parks[['num_st', 'city', 'state', 'phone_num']] = list_of_parks['park_address'].str.split(',', n = 3, expand=True)
#the park_address column in the OG dataframe contains the phone number for the park, which would confuse the geocoding system. Here, I separate that column on comma delimiter. 

In [13]:
list_of_parks.head(10)

,park_name,county,park_identifier,park_address,fire_authority,jurisdiction,mh_spaces,rv_lots_drains,rv_lots_no_drains,operated_by,operater_address,num_st,city,state,phone_num
0,SPANISH RANCH II,ALAMEDA,01-0001-MP,"121 Ranchero Way, Hayward, Ca 94544, (510) 886...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",187.0,0.0,0.0,"HOMETOWN SPANISH RANCH, LLC, A DELAWARE LIMITE...","121 RANCHERO WAY, HAYWARD, CA 94544, (510) 886...",121 Ranchero Way,Hayward,Ca 94544,(510) 886-4646
1,GEORGIAN MANOR MHP,ALAMEDA,01-0003-MP,"1419 Buckingham Way, Hayward, Ca 94544, (510) ...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",265.0,0.0,0.0,"GEORGIAN MANOR MOBILEHOME PARK L P, 321 HARTZ ...","1419 BUCKINGHAM WAY, HAYWARD, CA 94544, (510) ...",1419 Buckingham Way,Hayward,Ca 94544,(510) 785-2212
2,BAL TRAILER CT,ALAMEDA,01-0008-MP,"14831 Bancroft Ave, San Leandro, Ca 94578, (51...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",31.0,0.0,0.0,"PUEBLO SPRINGS MHP LTD, 10351 SANTA MONICA BLV...","14831 BANCROFT AVE, SAN LEANDRO, CA 94578, (51...",14831 Bancroft Ave,San Leandro,Ca 94578,(510) 352-8152
3,VASCO MHP,ALAMEDA,01-0010-MP,"6539 S Front Rd, Livermore, Ca 94550, (925) 44...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",50.0,21.0,0.0,"SOLOMON/MANDEL, EDWARD/GEORGE, PO BOX 406, ALA...","6539 S FRONT RD, LIVERMORE, CA 94550, (925) 44...",6539 S Front Rd,Livermore,Ca 94550,(925) 447-0758
4,AVALON MOBILE HOME PARK,ALAMEDA,01-0013-MP,"3970 Castro Valley Blvd, Castro Valley, Ca 945...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",49.0,0.0,0.0,"AVALON MHP INC, 4061 EAST CASTRO VALLEY BLVD S...","3970 CASTRO VALLEY BLVD, CASTRO VALLEY, CA 945...",3970 Castro Valley Blvd,Castro Valley,Ca 94546,(510) 537-4815
5,NEW ENGLAND VILLAGE,ALAMEDA,01-0015-MP,"940 New England Village Dr, Hayward, Ca 94544,...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",415.0,0.0,0.0,"BRANDENBURG STAEDLER & MOORE, 1122 WILLOW ST S...","940 NEW ENGLAND VILLAGE DR, HAYWARD, CA 94544,...",940 New England Village Dr,Hayward,Ca 94544,(510) 785-4511
6,DEL RIO MOBILE HOME,ALAMEDA,01-0016-MP,"1550 162Nd Ave, San Leandro, Ca 94578, (510) 2...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",52.0,0.0,0.0,"CALIFANO FAMILY TRUST, 1550 162ND AVENUE SPACE...","1550 162ND AVE, SAN LEANDRO, CA 94578, (510) 2...",1550 162Nd Ave,San Leandro,Ca 94578,(510) 278-8810
7,DEL VALLE MHP,ALAMEDA,01-0017-MP,"1148 Arroyo Rd, Livermore, Ca 94550, (510) 331...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",29.0,0.0,0.0,"ABBOUD FAMILY TRUST, PO BOX 3975, HAYWARD, CA ...","1148 ARROYO RD, LIVERMORE, CA 94550, (510) 331...",1148 Arroyo Rd,Livermore,Ca 94550,(510) 331-2000
8,BAYSHORE COMMONS,ALAMEDA,01-0023-MP,"1468 Grand Ave, San Leandro, Ca 94577, (510) 3...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",40.0,0.0,0.0,"SAN LEANDRO TRAILER PARK LLC, 6653 EMBARCADERO...","1468 GRAND AVE, SAN LEANDRO, CA 94577, (510) 3...",1468 Grand Ave,San Leandro,Ca 94577,(510) 351-5950
9,FAIRVIEW TP,ALAMEDA,01-0024-MP,"785 Rose Ave, Pleasanton, Ca 94566, (925) 846-...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...","HCD - NORTHERN AREA OFFICE, 9342 TECH CENTER D...",14.0,8.0,0.0,"GIL, EDMUND, 2606 DERBY DR, SAN RAMON, CA 94583","785 ROSE AVE, PLEASANTON, CA 94566, (925) 846-...",785 Rose Ave,Pleasanton,Ca 94566,(925) 846-5074


In [14]:
list_of_parks['city'] = list_of_parks['city'].str.lstrip(' ')

In [15]:
list_of_parks[['state_only', 'zip']] = list_of_parks['state'].str.split(expand=True)

In [16]:
list_of_parks['state_only'] = list_of_parks.state_only.str.upper()

In [17]:
list_of_parks["updated_address"] = list_of_parks["num_st"] +","+ list_of_parks["city"] + "," + list_of_parks["state"]
#combine the relevant address columns into an updated address column

**Just in case it ends up being helpful, I will create a column of mobile home lot ranges.**

In [18]:
bins = [0,50, 100, 150, 200, 250, 300, 350, 400, 450, 500, np.inf]
labels = ["'0-50'",
          "'51-100'", 
          "'101-150'", 
          "'151-200'",
          "'201-250'",
          "'251-300'",
          "'301-350'",
          "'351-400'",
          "401-450'",
          "451-500'",
          "'>500'"]
list_of_parks["mh_spaces_ranges"] = pd.cut(list_of_parks["mh_spaces"], bins=bins, labels=labels)
#create new column with range of MH_Spaces

**Next, I will explore some of the data fields. First off, it looks like there are 57 counties which make sense. I will go through them and ensure there are no duplicates.**

In [19]:
list_of_parks["county"] = list_of_parks["county"].str.lower()

In [20]:
list_of_parks.county.nunique()

57

In [21]:
list_of_parks.county.unique()

array(['alameda', 'alpine', 'amador', 'butte', 'calaveras', 'colusa',
       'contra costa', 'del norte', 'el dorado', 'fresno', 'glenn',
       'humboldt', 'imperial', 'los angeles', 'riverside', 'inyo', 'kern',
       'kings', 'lake', 'lassen', 'madera', 'marin', 'mariposa',
       'tuolumne', 'merced', 'mendocino', 'modoc', 'mono', 'monterey',
       'santa cruz', 'napa', 'nevada', 'placer', 'orange', 'plumas',
       'sacramento', 'solano', 'san benito', 'san bernardino',
       'san diego', 'san joaquin', 'san luis obispo', 'san mateo',
       'santa barbara', 'santa clara', 'shasta', 'siskiyou', 'sierra',
       'sonoma', 'stanislaus', 'sutter', 'yolo', 'tehama', 'trinity',
       'tulare', 'ventura', 'yuba'], dtype=object)

**I'll also explore the city names, of which there are many.** 

In [22]:
list_of_parks.city.unique()

array(['Hayward', 'San Leandro', 'Livermore', 'Castro Valley',
       'Pleasanton', 'Oakland', 'Union City', 'Fremont', 'Markleeville',
       'Pine Grove', 'Ione', 'Pioneer', 'Martell', 'Jackson', 'Plymouth',
       'Oroville', 'Paradise', 'Chico', 'Gridley', 'Palermo',
       'Forbestown', 'Forest Ranch', 'Cohasset', 'Biggs', 'Magalia',
       'West Point', 'San Andreas', 'Avery', 'Valley Springs',
       'Angels Camp', 'Copperopolis', 'Dorrington', 'Railroad Flat',
       'Murphys', 'Williams', 'Colusa', 'Grimes', 'Maxwell', 'Arbuckle',
       '3 Mi E Of Williams', 'Stonyford', 'Oakley', 'San Pablo',
       'Bay Point', 'Antioch', 'El Sobrante', 'Pittsburg',
       'Bethel Island', 'Brentwood', 'Pacheco', 'Rodeo', 'Concord',
       'Pleasant Hill', 'Byron', 'Clayton', 'Crescent City',
       'Smith River', 'Klamath', 'Gasquet', 'Diamond Springs',
       'South Lake Tahoe', 'Placerville', 'Pollock Pines',
       'Shingle Springs', 'Camino', 'El Dorado', 'Georgetown', 'Lotus',
       

In [50]:
list_of_parks.city.nunique()

821

**There are some funky values in this column, like those that give a distance from the highway or one that is just named '#9.' Those might be an outcome of the split I did on the original address column.**

**Next, I'll filter this dataframe down so it only contains columns necessary to my analysis.**

In [23]:
list_of_parks_filtered = list_of_parks[['park_name',
                                        'mh_spaces',
                                        'city']]

In [24]:
list_of_parks_filtered.head()

,park_name,mh_spaces,city
0,SPANISH RANCH II,187.0,Hayward
1,GEORGIAN MANOR MHP,265.0,Hayward
2,BAL TRAILER CT,31.0,San Leandro
3,VASCO MHP,50.0,Livermore
4,AVALON MOBILE HOME PARK,49.0,Castro Valley


**The last thing I'll do is apply a groupby to the dataframe so there are columns for each city and the amount of MH spaces in each city.**

In [25]:
list_of_parks_analysis = list_of_parks_filtered.groupby('city').mh_spaces.sum().reset_index().sort_values('mh_spaces')

In [26]:
list_of_parks_analysis

,city,mh_spaces
128,Canyon Lake,1.0
33,Aromas,1.0
448,Magalia,1.0
594,Point Arena,1.0
420,Loch Lomond,1.0
...,...,...
653,San Diego,4747.0
232,El Cajon,5359.0
644,Sacramento,6729.0
322,Hemet,6987.0


**Export dataframe as CSV.**

In [27]:
outname = 'mh_parks_analysis.csv'
fullname = os.path.join(processed_outdir, outname)
list_of_parks_analysis.to_csv(fullname)

### Cleaning population dataset

In [51]:
census_pop_est = pd.read_csv(raw_dir + 'california_pop_est.csv', encoding='latin1') #latin1 bc csv isn't utf-8 encoded
census_pop_est

,SUMLEV,STATE,COUNTY,PLACE,COUSUB,CONCIT,PRIMGEO_FLAG,FUNCSTAT,NAME,STNAME,...,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE042020,POPESTIMATE2020
0,40,6,0,0,0,0,0,A,California,California,...,37944551,38253768,38586706,38904296,39149186,39337785,39437463,39437610,39418894,39368078
1,162,6,0,296,0,0,0,A,Adelanto city,California,...,31222,31414,32745,33376,33484,34309,34411,34506,34540,34537
2,162,6,0,394,0,0,0,A,Agoura Hills city,California,...,20484,20537,20585,20634,20586,20471,20307,20103,19954,19881
3,162,6,0,562,0,0,0,A,Alameda city,California,...,75750,76616,77646,78997,80023,80178,80196,79813,79170,78841
4,162,6,0,674,0,0,0,A,Albany city,California,...,19041,19286,19557,19827,19958,20168,20352,20257,20110,20031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072,157,6,113,99990,0,0,1,F,Balance of Yolo County,California,...,26132,26480,26138,26964,28477,29296,29636,29772,29402,29337
1073,50,6,115,0,0,0,0,A,Yuba County,California,...,72770,73082,73527,74039,74920,76575,77418,78549,79800,80160
1074,157,6,115,46170,0,0,1,A,Marysville city,California,...,12119,12169,12200,12212,12259,12453,12499,12526,12590,12594
1075,157,6,115,85012,0,0,1,A,Wheatland city,California,...,3525,3526,3533,3539,3559,3622,3647,3654,3665,3666


In [29]:
census_pop_est.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077 entries, 0 to 1076
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   SUMLEV             1077 non-null   int64 
 1   STATE              1077 non-null   int64 
 2   COUNTY             1077 non-null   int64 
 3   PLACE              1077 non-null   int64 
 4   COUSUB             1077 non-null   int64 
 5   CONCIT             1077 non-null   int64 
 6   PRIMGEO_FLAG       1077 non-null   int64 
 7   FUNCSTAT           1077 non-null   object
 8   NAME               1077 non-null   object
 9   STNAME             1077 non-null   object
 10  CENSUS2010POP      1077 non-null   object
 11  ESTIMATESBASE2010  1077 non-null   int64 
 12  POPESTIMATE2010    1077 non-null   int64 
 13  POPESTIMATE2011    1077 non-null   int64 
 14  POPESTIMATE2012    1077 non-null   int64 
 15  POPESTIMATE2013    1077 non-null   int64 
 16  POPESTIMATE2014    1077 non-null   int64 


In [30]:
census_pop_est.columns= census_pop_est.columns.str.lower()

In [31]:
census_pop_est.name.nunique()

595

**This is confusing - there are only 595 unique values under the name column but 1,077 observations. 595*2 != 1,077, so it's not as simple as there being duplicates of each observation. I'll try to figure out what's going on in the following cells.**

In [32]:
census_pop_est[census_pop_est.name.str.contains(" city| town")].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 964 entries, 1 to 1075
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   sumlev             964 non-null    int64 
 1   state              964 non-null    int64 
 2   county             964 non-null    int64 
 3   place              964 non-null    int64 
 4   cousub             964 non-null    int64 
 5   concit             964 non-null    int64 
 6   primgeo_flag       964 non-null    int64 
 7   funcstat           964 non-null    object
 8   name               964 non-null    object
 9   stname             964 non-null    object
 10  census2010pop      964 non-null    object
 11  estimatesbase2010  964 non-null    int64 
 12  popestimate2010    964 non-null    int64 
 13  popestimate2011    964 non-null    int64 
 14  popestimate2012    964 non-null    int64 
 15  popestimate2013    964 non-null    int64 
 16  popestimate2014    964 non-null    int64 
 

**Ok, so there are 964 entries under name that have city or town in the name.** 

In [33]:
census_pop_est[census_pop_est.name.str.contains(" County")].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112 entries, 483 to 1076
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   sumlev             112 non-null    int64 
 1   state              112 non-null    int64 
 2   county             112 non-null    int64 
 3   place              112 non-null    int64 
 4   cousub             112 non-null    int64 
 5   concit             112 non-null    int64 
 6   primgeo_flag       112 non-null    int64 
 7   funcstat           112 non-null    object
 8   name               112 non-null    object
 9   stname             112 non-null    object
 10  census2010pop      112 non-null    object
 11  estimatesbase2010  112 non-null    int64 
 12  popestimate2010    112 non-null    int64 
 13  popestimate2011    112 non-null    int64 
 14  popestimate2012    112 non-null    int64 
 15  popestimate2013    112 non-null    int64 
 16  popestimate2014    112 non-null    int64 

**There are 112 observations that have the word ' County' in name column.**

**112 + 964 = 1,076. Then there's one row for all of California, which equals the total amount of rows in the whole dataset. So I think I can confidently extract cities from this dataset by trying to find those with city or town in them.** 

In [34]:
ca_cities = census_pop_est[census_pop_est.name.str.contains(" city| town")]

In [35]:
ca_cities.name.nunique()

482

**Ok so the number of unique names in the cities is half of the entries in the cities dataframe. That must mean each is repeated twice. That would mean the reason there are 1,077 observations but only 595 unique values under 'name' is that each city/town is duplicated but each county isn't.**

**Let me try to figure see if there are major differences in the rows that are duplicated out by selecting specific cities.**

In [36]:
ca_cities[ca_cities.name == 'Paradise town']

,sumlev,state,county,place,cousub,concit,primgeo_flag,funcstat,name,stname,...,popestimate2012,popestimate2013,popestimate2014,popestimate2015,popestimate2016,popestimate2017,popestimate2018,popestimate2019,popestimate042020,popestimate2020
310,162,6,0,55520,0,0,0,A,Paradise town,California,...,26036,26047,26191,26210,26289,26469,26532,4171,4294,4329
512,157,6,7,55520,0,0,1,A,Paradise town,California,...,26036,26047,26191,26210,26289,26469,26532,4171,4294,4329


**Everything looks the same other than the value under primgeo_flag.** 

**Let me try the same thing with a different city name.** 

In [37]:
ca_cities[ca_cities.name == 'Palo Alto city']

,sumlev,state,county,place,cousub,concit,primgeo_flag,funcstat,name,stname,...,popestimate2012,popestimate2013,popestimate2014,popestimate2015,popestimate2016,popestimate2017,popestimate2018,popestimate2019,popestimate042020,popestimate2020
308,162,6,0,55282,0,0,0,A,Palo Alto city,California,...,66193,66982,67459,68125,68599,68513,68136,67621,67238,67008
970,157,6,85,55282,0,0,1,A,Palo Alto city,California,...,66193,66982,67459,68125,68599,68513,68136,67621,67238,67008


**That one is also the same except the primgeo_flag. According to the [data dictionary]('https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2020/sub-est2020.pdf'), this column refers to the primitive geography level.** 

**According to the Census [website]('https://www.census.gov/newsroom/blogs/random-samplings/2017/10/city_and_town_orsu.html'), the term refers to the 'smallest pieces of land that make up counties, cities and towns which cannot be further subdivided.'** 

**Let me do one more check with city that's bigger.**

In [38]:
ca_cities[ca_cities.name == 'Los Angeles city']

,sumlev,state,county,place,cousub,concit,primgeo_flag,funcstat,name,stname,...,popestimate2012,popestimate2013,popestimate2014,popestimate2015,popestimate2016,popestimate2017,popestimate2018,popestimate2019,popestimate042020,popestimate2020
241,162,6,0,44000,0,0,0,A,Los Angeles city,California,...,3847857,3877721,3904102,3933644,3957520,3975067,3981140,3982885,3976825,3970219
666,157,6,37,44000,0,0,1,A,Los Angeles city,California,...,3847857,3877721,3904102,3933644,3957520,3975067,3981140,3982885,3976825,3970219


**I feel comfortable getting rid of the duplicates after seeing that three different cities of different populations have the same numbers despite having different values under the primgeo_flag colum.**

**I'll also change the name of the 'name' column to 'city' so that it's consistent with the dataframe on mobile home spaces.**

In [39]:
ca_cities = ca_cities[ca_cities.primgeo_flag != 1]
ca_cities.rename(columns={'name':'city'}, inplace=True)
ca_cities.info()
ca_cities.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 482 entries, 1 to 482
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   sumlev             482 non-null    int64 
 1   state              482 non-null    int64 
 2   county             482 non-null    int64 
 3   place              482 non-null    int64 
 4   cousub             482 non-null    int64 
 5   concit             482 non-null    int64 
 6   primgeo_flag       482 non-null    int64 
 7   funcstat           482 non-null    object
 8   city               482 non-null    object
 9   stname             482 non-null    object
 10  census2010pop      482 non-null    object
 11  estimatesbase2010  482 non-null    int64 
 12  popestimate2010    482 non-null    int64 
 13  popestimate2011    482 non-null    int64 
 14  popestimate2012    482 non-null    int64 
 15  popestimate2013    482 non-null    int64 
 16  popestimate2014    482 non-null    int64 
 1

,sumlev,state,county,place,cousub,concit,primgeo_flag,funcstat,city,stname,...,popestimate2012,popestimate2013,popestimate2014,popestimate2015,popestimate2016,popestimate2017,popestimate2018,popestimate2019,popestimate042020,popestimate2020
1,162,6,0,296,0,0,0,A,Adelanto city,California,...,31222,31414,32745,33376,33484,34309,34411,34506,34540,34537
2,162,6,0,394,0,0,0,A,Agoura Hills city,California,...,20484,20537,20585,20634,20586,20471,20307,20103,19954,19881
3,162,6,0,562,0,0,0,A,Alameda city,California,...,75750,76616,77646,78997,80023,80178,80196,79813,79170,78841
4,162,6,0,674,0,0,0,A,Albany city,California,...,19041,19286,19557,19827,19958,20168,20352,20257,20110,20031
5,162,6,0,884,0,0,0,A,Alhambra city,California,...,84006,84305,84635,85086,85341,85270,84666,83932,83502,83261


**So that I'm able to merge with the other dataframe, I'll also remove the 'city' and 'town' substrings that come after each entry.**

In [40]:
ca_cities['city'] = ca_cities['city'].str.replace(' city','')

In [41]:
ca_cities['city'] = ca_cities['city'].str.replace(' town', '')

**I want to look specifically at the 2018 population estimate, because any year thereafter will take into account the population exodus that happened in Paradise after the fire.** 

In [42]:
ca_cities_2018_pop = ca_cities[['city',
                            'popestimate2018']]

In [43]:
ca_cities_2018_pop

,city,popestimate2018
1,Adelanto,34411
2,Agoura Hills,20307
3,Alameda,80196
4,Albany,20352
5,Alhambra,84666
...,...,...
478,Yountville,3002
479,Yreka,7594
480,Yuba City,66654
481,Yucaipa,54482


**Export the dataframe to my processed directory.**

In [44]:
outname = 'ca_cities_2018_analysis.csv'
fullname = os.path.join(processed_outdir, outname)
ca_cities_2018_pop.to_csv(fullname)

### Merging

**In the following steps, I'll do a left join on the two dataframes on the common column of 'city.' Then I'll export this into my processed data directory.**

In [45]:
merged = pd.merge(ca_cities_2018_pop, list_of_parks_analysis, how='left', on=['city', 'city'])

In [46]:
pd.set_option('display.max_rows', 500)

In [47]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 482 entries, 0 to 481
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   city             482 non-null    object 
 1   popestimate2018  482 non-null    int64  
 2   mh_spaces        372 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 15.1+ KB


In [48]:
merged

,city,popestimate2018,mh_spaces
0,Adelanto,34411,257.0
1,Agoura Hills,20307,NaN
2,Alameda,80196,NaN
3,Albany,20352,NaN
4,Alhambra,84666,NaN
5,Aliso Viejo,50739,NaN
6,Alturas,2530,67.0
7,Amador City,196,NaN
8,American Canyon,20412,843.0
9,Anaheim,353043,3542.0


In [49]:
outname = 'merged.csv'
fullname = os.path.join(processed_outdir, outname)
merged.to_csv(fullname)

**There are only 372 matches between the two datasets, so I'll do a right join and see which cities from the MH dataframe were dropped in the left merge.**

In [52]:
merged_right = pd.merge(ca_cities_2018_pop, list_of_parks_analysis, how='right', on=['city', 'city'])

In [62]:
null_cities = merged_right[merged_right['popestimate2018'].isnull()]

**As can be expected, some of the funky names in the MH databse didn't make the left merge, but there are some, like Ventura and Paso Robles, that you'd expect should be a match.**

In [63]:
null_cities

,city,popestimate2018,mh_spaces
1,Aromas,NaN,1.0
2,Magalia,NaN,1.0
4,Loch Lomond,NaN,1.0
5,Prattville,NaN,1.0
6,Holt,NaN,1.0
7,Beckwourth,NaN,1.0
8,Scotia,NaN,1.0
9,Vernalis,NaN,1.0
10,Duncans Mills,NaN,1.0
11,Creston,NaN,1.0


**From a quick search, I see that Ventura and Paso Robles are in the Census dataset but under different names. For example, Ventura is listed in the Census dataset as "San Buenaventura (Ventura)" and Paso Robles is "El Paso de Robles (Paso Robles)."**
